In [1]:
main_dir = '/Users/simon/fbso/fbso/bsose_monthly/'
salt = main_dir + 'bsose_i106_2008to2012_monthly_Salt.nc'
theta = main_dir + 'bsose_i106_2008to2012_monthly_Theta.nc'

In [6]:
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import pyxpcm
from pyxpcm.models import pcm


def pcm_fit_and_post(time_i=42, K=4, maxvar=15):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(0., -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=time_i)
    theta_nc = xr.open_dataset(theta).isel(time=time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > max_depth).drop(['iter', 'Depth', 'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
            
    ds = both_nc # .copy(deep=True)
    m = pcm(K=K, features=features_pcm, maxvar=maxvar)
    m.fit(ds, features=features, dim='Z') #, inplace=True)
    m.predict(ds, features=features, dim='Z', inplace=True)
    m.predict_proba(ds, features=features, dim='Z', inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    return ds, m


def plot_pcm_labels(xr_ds):

    map_proj = ccrs.SouthPolarStereo()
    carree = ccrs.PlateCarree()

    fig = plt.figure(figsize=[5, 5])

    ax1 = plt.subplot(1, 1, 1, projection=map_proj)
    fig.subplots_adjust(bottom=0.05, top=0.95,
                        left=0.04, right=0.95, wspace=0.02)
    ax1.set_extent([-180, 180, -90, -30], carree)
    
    def plot_boundary():
        theta = np.linspace(0, 2 * np.pi, 100)
        center, radius = [0.5, 0.5], 0.45
        verts = np.vstack([np.sin(theta), np.cos(theta)]).T
        circle = mpath.Path(verts * radius + center)
        ax1.set_boundary(circle, transform=ax1.transAxes)
    
    plot_boundary()
    

    xr_ds.PCM_LABELS.plot(transform=carree,                      # the data's projection
                          ax=ax1,
                          subplot_kws={"projection": map_proj},  # the plot's projection
                          cbar_kwargs={"shrink": 0.3},
                          )
    
    ax1.coastlines()
    
    ax1.gridlines(xlocs=range(0, 361, 45), ylocs=range(-80, -19, 20))
    
    plt.show()
    

In [3]:
ds, m = pcm_fit_and_post(K=5)
ds

<xarray.Dataset>
Dimensions:     (XC: 2160, YC: 588, Z: 52, pcm_class: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * XC          (XC) float64 0.08333 0.25 0.4167 0.5833 ... 359.6 359.8 359.9
  * YC          (YC) float64 -77.98 -77.95 -77.91 ... -30.02 -29.87 -29.72
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
Dimensions without coordinates: pcm_class
Data variables:
    SALT        (Z, YC, XC) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCM_LABELS  (YC, XC) float64 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCM_POST    (pcm_class, YC, XC) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0

In [7]:
ds1, m1 = pcm_fit_and_post(K=5, maxvar=0.6)
ds1

<xarray.Dataset>
Dimensions:     (XC: 2160, YC: 588, Z: 52, pcm_class: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * XC          (XC) float64 0.08333 0.25 0.4167 0.5833 ... 359.6 359.8 359.9
  * YC          (YC) float64 -77.98 -77.95 -77.91 ... -30.02 -29.87 -29.72
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
Dimensions without coordinates: pcm_class
Data variables:
    SALT        (Z, YC, XC) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCM_LABELS  (YC, XC) float64 nan nan nan nan nan nan ... 1.0 1.0 1.0 1.0 1.0
    PCM_POST    (pcm_class, YC, XC) float64 nan nan ... 8.046e-102 1.404e-101

In [8]:
m1

<pcm 'gmm' (K: 5, F: 2)>
Number of class: 5
Number of feature: 2
Feature names: odict_keys(['THETA', 'SALT'])
Fitted: True
Feature: 'THETA'
	 Interpoler: <class 'pyxpcm.utils.Vertical_Interpolator'>
	 Scaler: 'normal', <class 'sklearn.preprocessing._data.StandardScaler'>
	 Reducer: True, <class 'sklearn.decomposition._pca.PCA'>
Feature: 'SALT'
	 Interpoler: <class 'pyxpcm.utils.Vertical_Interpolator'>
	 Scaler: 'normal', <class 'sklearn.preprocessing._data.StandardScaler'>
	 Reducer: True, <class 'sklearn.decomposition._pca.PCA'>
Classifier: 'gmm', <class 'sklearn.mixture._gaussian_mixture.GaussianMixture'>
	 log likelihood of the training set: -0.928730

In [10]:
m1._reducer

OrderedDict([('THETA',
              PCA(copy=True, iterated_power='auto', n_components=0.6, random_state=None,
                  svd_solver='full', tol=0.0, whiten=False)),
             ('SALT',
              PCA(copy=True, iterated_power='auto', n_components=0.6, random_state=None,
                  svd_solver='full', tol=0.0, whiten=False))])

In [11]:
m1._context

<bound method pcm.__empty_context of <pcm 'gmm' (K: 5, F: 2)>
Number of class: 5
Number of feature: 2
Feature names: odict_keys(['THETA', 'SALT'])
Fitted: True
Feature: 'THETA'
	 Interpoler: <class 'pyxpcm.utils.Vertical_Interpolator'>
	 Scaler: 'normal', <class 'sklearn.preprocessing._data.StandardScaler'>
	 Reducer: True, <class 'sklearn.decomposition._pca.PCA'>
Feature: 'SALT'
	 Interpoler: <class 'pyxpcm.utils.Vertical_Interpolator'>
	 Scaler: 'normal', <class 'sklearn.preprocessing._data.StandardScaler'>
	 Reducer: True, <class 'sklearn.decomposition._pca.PCA'>
Classifier: 'gmm', <class 'sklearn.mixture._gaussian_mixture.GaussianMixture'>
	 log likelihood of the training set: -0.928730>

In [13]:
m._reducer

OrderedDict([('THETA',
              PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
                  svd_solver='full', tol=0.0, whiten=False)),
             ('SALT',
              PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
                  svd_solver='full', tol=0.0, whiten=False))])

In [15]:
ds2, m2 = pcm_fit_and_post(K=5, maxvar=30)
ds2

<xarray.Dataset>
Dimensions:     (XC: 2160, YC: 588, Z: 52, pcm_class: 5)
Coordinates:
    time        datetime64[ns] 2011-08-01T15:12:00
  * XC          (XC) float64 0.08333 0.25 0.4167 0.5833 ... 359.6 359.8 359.9
  * YC          (YC) float64 -77.98 -77.95 -77.91 ... -30.02 -29.87 -29.72
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
Dimensions without coordinates: pcm_class
Data variables:
    SALT        (Z, YC, XC) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA       (Z, YC, XC) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    PCM_LABELS  (YC, XC) float64 nan nan nan nan nan nan ... 2.0 2.0 2.0 2.0 2.0
    PCM_POST    (pcm_class, YC, XC) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0

In [ ]:
pcm = xr.open_dataset('pcm.nc')